In [1]:
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices=false'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
import optuna
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.regularizers import l2, l1_l2
import matplotlib.pyplot as plt
import gc

2025-06-01 08:43:24.425393: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748778204.547636 3723708 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748778204.582478 3723708 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748778204.849439 3723708 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748778204.849475 3723708 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748778204.849477 3723708 computation_placer.cc:177] computation placer alr

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Tratamento dos Dados

In [ ]:
df = pd.read_csv('../dataset_fz_cz.csv')

df

In [ ]:
fft_columns = df.iloc[:, :1500]
def safe_complex(x):
    try:
        return complex(x)
    except (ValueError, TypeError):
        return np.nan
fft_complex = fft_columns.map(safe_complex)
df.iloc[:, :1500] = fft_complex

## Separando X e Y

In [ ]:
y = df['classe']
X = df.drop(['classe','voluntario'], axis=1)

In [ ]:
X

In [ ]:
y.value_counts()

## Divisão em conjunto de Treino e Teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
X_train_rec_f0 = X_train.iloc[:, :500]
X_test_rec_f0 = X_test.iloc[:, :500]

In [ ]:
X_train.iloc[:,:1500] = X_train.iloc[:,:1500].map(lambda x: np.abs(x))
X_test.iloc[:,:1500] = X_test.iloc[:,:1500].map(lambda x: np.abs(x))

In [ ]:
X_train.shape, X_test.shape

In [ ]:
X_train

In [ ]:
indice_f0 = list(range(500)) + [1500] + list(range(1503, X_train.shape[1]))
X_train_f0 = X_train.iloc[:, indice_f0]
X_test_f0 = X_test.iloc[:, indice_f0]

In [ ]:
indice_h2 = list(range(1000)) + [1500, 1501] + list(range(1503, X_train.shape[1]))
X_train_h2 = X_train.iloc[:, indice_h2]
X_test_h2 = X_test.iloc[:, indice_h2]

## Codificar Variáveis Categóricas

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_columns = ['Faixa_Moduladora', 
                     'Intensidade_db', 
                     'Canal']

numerical_columns = ['Freq_analisada_Hz']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(), categorical_columns)
    ],
    remainder='passthrough'
)

X_train_f0 = preprocessor.fit_transform(X_train_f0)
X_test_f0 = preprocessor.transform(X_test_f0)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_columns = ['Faixa_Moduladora', 
                     'Intensidade_db', 
                     'Canal']

numerical_columns = ['Freq_analisada_Hz', 'Freq_h2_analisada_Hz',]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(), categorical_columns)
    ],
    remainder='passthrough'
)

X_train_h2 = preprocessor.fit_transform(X_train_h2)
X_test_h2 = preprocessor.transform(X_test_h2)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_columns = ['Faixa_Moduladora', 
                     'Intensidade_db', 
                     'Canal']

numerical_columns = ['Freq_analisada_Hz', 
                     'Freq_h2_analisada_Hz',
                     'Freq_h3_analisada_Hz']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(), categorical_columns)
    ],
    remainder='passthrough'
)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

Com o ColumnTransformer, as colunas após o pré-processamento podem ter nomes alterados (ex.: Faixa_40, Faixa_80). 

In [ ]:
colunas_processadas = preprocessor.get_feature_names_out()

In [ ]:
colunas_processadas

In [ ]:
pd.DataFrame(X_train).head()

In [ ]:
X_train

In [ ]:
pd.DataFrame(X_train).to_parquet('f0_h2_h3/X_train.parquet', index=False)
pd.DataFrame(X_test).to_parquet('f0_h2_h3/X_test.parquet', index=False)
pd.DataFrame(y_train).to_parquet('f0_h2_h3/y_train.parquet', index=False)
pd.DataFrame(y_test).to_parquet('f0_h2_h3/y_test.parquet', index=False)

In [ ]:
X_train_rec_f0.to_csv('f0/X_train_rec_f0.csv', index=False)
X_test_rec_f0.to_csv('f0/X_test_rec_f0.csv', index=False)

In [ ]:
pd.DataFrame(X_train_f0).to_parquet('f0/X_train_f0.parquet', index=False)
pd.DataFrame(X_test_f0).to_parquet('f0/X_test_f0.parquet', index=False)
pd.DataFrame(y_train).to_parquet('f0/y_train_f0.parquet', index=False)
pd.DataFrame(y_test).to_parquet('f0/y_test_f0.parquet', index=False)

In [ ]:
pd.DataFrame(X_train_h2).to_parquet('f0_h2/X_train_h2.parquet', index=False)
pd.DataFrame(X_test_h2).to_parquet('f0_h2/X_test_h2.parquet', index=False)
pd.DataFrame(y_train).to_parquet('f0_h2/y_train_h2.parquet', index=False)
pd.DataFrame(y_test).to_parquet('f0_h2/y_test_h2.parquet', index=False)

Até aqui, foi realizado a separação do X e Y, nos quais foram separados em conjunto de teste e treinamento. Além disso, os conjuntos de treino para os modelos de ML foram transformados para pegar apenas os módulos, já os em formato retangular, será utilizado na CSM e MSC. Obtendo assim, 4 datasets finais

# Desenvolvimento dos Modelos

## Carregando os Dados

In [4]:
X_train_f0 = pd.read_parquet('f0/X_train_f0.parquet')
X_test_f0 = pd.read_parquet('f0/X_test_f0.parquet')
y_train_f0 = pd.read_parquet('f0/y_train_f0.parquet')
y_test_f0 = pd.read_parquet('f0/y_test_f0.parquet')

In [5]:
X_train_h2 = pd.read_parquet('f0_h2/X_train_h2.parquet')
X_test_h2 = pd.read_parquet('f0_h2/X_test_h2.parquet')
y_train_h2 = pd.read_parquet('f0_h2/y_train_h2.parquet')
y_test_h2 = pd.read_parquet('f0_h2/y_test_h2.parquet')

In [6]:
X_train = pd.read_parquet('f0_h2_h3/X_train.parquet')
X_test = pd.read_parquet('f0_h2_h3/X_test.parquet')
y_train = pd.read_parquet('f0_h2_h3/y_train.parquet')
y_test = pd.read_parquet('f0_h2_h3/y_test.parquet')

In [7]:
datasets = {
    'Fundamental': {
        'X_train': X_train_f0,
        'y_train': y_train_f0,
        'X_test': X_test_f0,
        'y_test': y_test_f0
    },
    'Fundamental + 2º Harmônico': {
        'X_train': X_train_h2,
        'y_train': y_train_h2,
        'X_test': X_test_h2,
        'y_test': y_test_h2
    },
    'Fundamental + 2º + 3º Harmônicos': {
        'X_train': X_train,
        'y_train': y_train,
        'X_test': X_test,
        'y_test': y_test
    }
}

## Criando o modelo RNA

### RNA Padrão

In [ ]:
def create_model(trial, input_shape):
    n_layers = trial.suggest_int('n_layers', 1, 100)
    units = trial.suggest_int('units', 32, 512, step=32)
    dropout = trial.suggest_float('dropout', 0.0, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-6, 1e-1, log=True)
    
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=input_shape))
    for i in range(n_layers):
        model.add(layers.Dense(units, activation='relu'))
        model.add(layers.Dropout(dropout))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    return model

### RNA variando parametros por camada

In [8]:
def create_model_layer(trial, input_shape):
    n_layers = trial.suggest_int('n_layers', 1, 50)
    units = []
    dropouts = []

    for i in range(n_layers):
        units.append(trial.suggest_int(f'units_{i}', 32, 512, step=32))
        dropouts.append(trial.suggest_float(f'dropout_{i}', 0.0, 0.5))

    learning_rate = trial.suggest_float('learning_rate', 1e-6, 1e-1, log=True)
    l2_weight = trial.suggest_float('l2_weight', 1e-5, 1e-2, log=True)
    l1_weight = trial.suggest_float('l1_weight', 1e-5, 1e-2, log=True)
    
    model = models.Sequential()
    model.add(layers.InputLayer(shape=input_shape))
    
    for i in range(n_layers):
        # activation_function = trial.suggest_categorical(f'activation_{i}', ['relu', 'leaky_relu', 'elu'])
        # if activation_function == 'leaky_relu':
        #     model.add(layers.Dense(units[i], kernel_regularizer=l1_l2(l1=l1_weight, l2=l2_weight)))
        #     model.add(layers.LeakyReLU())
        # else:
        #     model.add(layers.Dense(units[i], activation=activation_function, kernel_regularizer=l1_l2(l1=l1_weight, l2=l2_weight)))
            
        model.add(layers.Dense(units[i], activation='relu', kernel_regularizer=l1_l2(l1=l1_weight, l2=l2_weight)))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(dropouts[i]))
    
    model.add(layers.Dense(1, activation='sigmoid'))

    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'RMSprop', 'SGD'])
    
    model.compile(
        # optimizer=tf.keras.optimizers.get({
        #     'class_name': optimizer_name,
        #     'config': {
        #         'learning_rate': learning_rate
        #     }
        # }),
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    return model

### RNA + LSTM

In [ ]:
def create_sequence(data, labels, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length + 1):
        X.append(data[i:i + seq_length])
        y.append(labels[i + seq_length - 1])
    return np.array(X), np.array(y)

X_train_f0_seq, y_train_f0_seq = create_sequence(X_train_f0.values, y_train_f0.values, 20)
X_test_f0_seq, y_test_f0_seq = create_sequence(X_test_f0.values, y_test_f0.values, 20)

X_train_h2_seq, y_train_h2_seq = create_sequence(X_train_h2.values, y_train_h2.values, 20)
X_test_h2_seq, y_test_h2_seq = create_sequence(X_test_h2.values, y_test_h2.values, 20)

X_train_seq, y_train_seq = create_sequence(X_train.values, y_train.values, 20)
X_test_seq, y_test_seq = create_sequence(X_test.values, y_test.values, 20)
        

In [ ]:
datasets_seq = {
    'Fundamental': {
        'X_train': X_train_f0_seq,
        'y_train': y_train_f0_seq,
        'X_test': X_test_f0_seq,
        'y_test': y_test_f0_seq
    },
    'Fundamental + 2º Harmônico': {
        'X_train': X_train_h2_seq,
        'y_train': y_train_h2_seq,
        'X_test': X_test_h2_seq,
        'y_test': y_test_h2_seq
    },
    'Fundamental + 2º + 3º Harmônicos': {
        'X_train': X_train_seq,
        'y_train': y_train_seq,
        'X_test': X_test_seq,
        'y_test': y_test_seq
    }
}

In [ ]:
def create_lstm_model(trial, input_shape):
    # Hiperparâmetros para LSTM
    n_lstm_layers = trial.suggest_int('n_lstm_layers', 1, 10)
    lstm_units = [trial.suggest_int(f'lstm_units_{i}', 32, 256) for i in range(n_lstm_layers)]
    lstm_dropout = [trial.suggest_float(f'lstm_dropout_{i}', 0.0, 0.5) for i in range(n_lstm_layers)]
    
    # Hiperparâmetros para Dense
    n_dense_layers = trial.suggest_int('n_dense_layers', 1, 50)
    dense_units = [trial.suggest_int(f'dense_units_{i}', 32, 512) for i in range(n_dense_layers)]
    dense_dropout = [trial.suggest_float(f'dense_dropout_{i}', 0.0, 0.5) for i in range(n_dense_layers)]
    
    # Construção do modelo
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=input_shape))  # (look_back, n_features)
    
    # Camadas LSTM
    for i in range(n_lstm_layers):
        return_sequences = (i < n_lstm_layers - 1)  # Retorna sequências se não for a última camada
        model.add(layers.LSTM(
            lstm_units[i],
            return_sequences=return_sequences,
            kernel_regularizer=l2(trial.suggest_float('l2_weight', 1e-5, 1e-2))
        ))
        model.add(layers.Dropout(lstm_dropout[i]))
    
    # Camadas Densas
    for i in range(n_dense_layers):
        model.add(layers.Dense(dense_units[i], activation='relu'))
        model.add(layers.Dropout(dense_dropout[i]))
    
    # Saída
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Compilar
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=trial.suggest_float('learning_rate', 1e-6, 1e-1, log=True)),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    return model

### CNN-LSTM

In [ ]:
def create_sequence(data, labels, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length + 1):
        X.append(data[i:i + seq_length])
        y.append(labels[i + seq_length - 1])
    return np.array(X), np.array(y)

X_train_f0_seq, y_train_f0_seq = create_sequence(X_train_f0.values, y_train_f0.values, 20)
X_test_f0_seq, y_test_f0_seq = create_sequence(X_test_f0.values, y_test_f0.values, 20)

X_train_h2_seq, y_train_h2_seq = create_sequence(X_train_h2.values, y_train_h2.values, 20)
X_test_h2_seq, y_test_h2_seq = create_sequence(X_test_h2.values, y_test_h2.values, 20)

X_train_seq, y_train_seq = create_sequence(X_train.values, y_train.values, 20)
X_test_seq, y_test_seq = create_sequence(X_test.values, y_test.values, 20)
        

In [ ]:
datasets_seq = {
    'Fundamental': {
        'X_train': X_train_f0_seq,
        'y_train': y_train_f0_seq,
        'X_test': X_test_f0_seq,
        'y_test': y_test_f0_seq
    },
    'Fundamental + 2º Harmônico': {
        'X_train': X_train_h2_seq,
        'y_train': y_train_h2_seq,
        'X_test': X_test_h2_seq,
        'y_test': y_test_h2_seq
    },
    'Fundamental + 2º + 3º Harmônicos': {
        'X_train': X_train_seq,
        'y_train': y_train_seq,
        'X_test': X_test_seq,
        'y_test': y_test_seq
    }
}

In [ ]:
def create_cnn_lstm_model(trial, input_shape):
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=input_shape))
    
    current_seq_length = input_shape[0] # Comprimento da sequência inicial

    n_cnn_layers = trial.suggest_int('n_cnn_layers', 1, 3) # Número de blocos CNN (Conv1D + MaxPool + Dropout)

    # Hiperparâmetros para as camadas CNN
    for i in range(n_cnn_layers):
        filters = trial.suggest_int(f'cnn_filters_{i}', 32, 256, step=32)
        
        # Sugere kernel_size DENTRO do loop, limitado pelo current_seq_length
        max_kernel_size = min(current_seq_length, 5) # Maximo 5, ou o que restar da sequência
        if max_kernel_size < 2: # Se for menor que 2, não podemos usar kernel_size 2 ou maior
            raise optuna.exceptions.TrialPruned(
                f"Comprimento da sequência ({current_seq_length}) é muito pequeno para kernel_size >= 2 na camada CNN {i+1}. Trial podado."
            )
        kernel_size = trial.suggest_int(f'cnn_kernel_size_{i}', 2, max_kernel_size)

        # Calcula o novo comprimento da sequência após a Conv1D (padding='valid')
        new_seq_length_after_conv = (current_seq_length - kernel_size) + 1
        
        # Sugere pool_size DENTRO do loop, limitado pelo new_seq_length_after_conv
        max_pooling_size = min(new_seq_length_after_conv, 4) # Maximo 4, ou o que restar após Conv1D
        if max_pooling_size < 2: # Se for menor que 2, não podemos usar pool_size 2 ou maior
            raise optuna.exceptions.TrialPruned(
                f"Comprimento da sequência após Conv1D ({new_seq_length_after_conv}) é muito pequeno para pool_size >= 2 na camada CNN {i+1}. Trial podado."
            )
        pool_size = trial.suggest_int(f'cnn_pooling_size_{i}', 2, max_pooling_size)
        
        cnn_dropout_rate = trial.suggest_float(f'cnn_dropout_{i}', 0.0, 0.5)

        model.add(layers.Conv1D(
            filters=filters,
            kernel_size=kernel_size,
            activation='relu',
            padding='valid' # Redução de dimensão
        ))
        model.add(layers.MaxPooling1D(pool_size=pool_size))
        model.add(layers.Dropout(cnn_dropout_rate))

        # Atualiza o comprimento da sequência para a próxima iteração do loop CNN
        current_seq_length = int(np.floor(new_seq_length_after_conv / pool_size))
        
        # Se o comprimento da sequência se tornar 0 ou negativo antes da última camada CNN, podar o trial
        if current_seq_length <= 0 and i < n_cnn_layers - 1:
            raise optuna.exceptions.TrialPruned(
                f"Comprimento da sequência se tornou <= 0 após a camada CNN {i+1}. Trial podado."
            )

    n_lstm_layers = trial.suggest_int('n_lstm_layers', 1, 5)
    lstm_units = [trial.suggest_int(f'lstm_units_{i}', 32, 256) for i in range(n_lstm_layers)]
    lstm_dropout = [trial.suggest_float(f'lstm_dropout_{i}', 0.0, 0.5) for i in range(n_lstm_layers)]
    
    n_dense_layers = trial.suggest_int('n_dense_layers', 1, 5)
    dense_units = [trial.suggest_int(f'dense_units_{i}', 32, 512) for i in range(n_dense_layers)]
    dense_dropout = [trial.suggest_float(f'dense_dropout_{i}', 0.0, 0.5) for i in range(n_dense_layers)]
    # Camadas LSTM
    for i in range(n_lstm_layers):
        return_sequences = (i < n_lstm_layers - 1)
        model.add(layers.LSTM(
            lstm_units[i],
            return_sequences=return_sequences,
            kernel_regularizer=l2(trial.suggest_float('l2_weight', 1e-5, 1e-2, log=True))
        ))
        model.add(layers.Dropout(lstm_dropout[i]))

    # Camadas Densas
    for i in range(n_dense_layers):
        model.add(layers.Dense(dense_units[i], activation='relu'))
        model.add(layers.Dropout(dense_dropout[i]))

    # Saída
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Compilar
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=trial.suggest_float('learning_rate', 1e-6, 1e-1, log=True)),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    return model
    

## Optuna

### Função Objetiva para todas as frequências

In [9]:
def objective(trial, X_train, y_train):
    input_shape = X_train.shape[1:]

    n_split = 5
    kf = StratifiedKFold(n_splits=n_split, shuffle=True, random_state=42)
    fold_aucs = []

    y_train_1d = y_train.squeeze().astype(int)
    classes = np.unique(y_train_1d)
    class_weights = compute_class_weight('balanced', classes=classes, y=y_train_1d)
    class_weight = {i: w for i, w in zip(classes, class_weights)}

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train_1d)):
        print(f"=== Iniciando Fold {fold + 1} de {n_split} ===")

        X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_train_1d[train_idx], y_train_1d[val_idx]

        model = create_model_layer(trial, input_shape)  # Mudar se quer RNA
        #model = create_lstm_model(trial, input_shape) #Mudar se quer RNA ou RNA + LSTM ou CNN + LSTM
        #model = create_cnn_lstm_model(trial, input_shape)  # CNN + LSTM

        callback = tf.keras.callbacks.EarlyStopping(
            monitor='val_auc',
            patience=30,
            verbose=0,
            mode='max',
            restore_best_weights=True
        )

        lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_auc',
            factor=0.5,
            patience=20,
            mode='max',
            min_lr=1e-7,
        )

        history = model.fit(
            X_train_fold,
            y_train_fold,
            epochs=100,
            batch_size=32,
            validation_data=(X_val_fold, y_val_fold),
            callbacks=[callback, lr_scheduler],
            verbose=1,
            class_weight=class_weight,
        )

        best_auc_fold = max(history.history['val_auc'])
        fold_aucs.append(best_auc_fold)
        print(f"AUC do Fold {fold + 1}: {best_auc_fold:.4f}")

        del model
        del callback, lr_scheduler, history
        tf.keras.backend.clear_session()
        gc.collect()
    print("=== Fim dos Folds ===")
    
    mean_auc = np.mean(fold_aucs)
    print(f"AUC Médio dos Folds: {mean_auc:.4f}")

    return mean_auc


### Treinando Modelo com os melhores hiperparametros - RNA

In [ ]:
def build_final_model(best_params, input_shape):
    n_layers = best_params['n_layers']
    units = best_params['units']
    dropout = best_params['dropout']
    learning_rate = best_params['learning_rate']
   
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=input_shape))
    for i in range(n_layers):
        model.add(layers.Dense(units, activation='relu'))
        model.add(layers.Dropout(dropout))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

In [10]:
def build_final_model_layer(best_params, input_shape):
    model = models.Sequential()
    model.add(layers.InputLayer(shape=input_shape))
    
    # Extrai hiperparâmetros por camada
    n_layers = best_params['n_layers']
    units = [best_params[f'units_{i}'] for i in range(n_layers)]
    dropouts = [best_params[f'dropout_{i}'] for i in range(n_layers)]
    
    # Constrói as camadas
    for i in range(n_layers):
        # activation_function = best_params.get(f'activation_{i}', 'relu')
        # if activation_function == 'leaky_relu':
        #     model.add(layers.Dense(units[i], kernel_regularizer=l1_l2(l1=best_params['l1_weight'], l2=best_params['l2_weight'])))
        #     model.add(layers.LeakyReLU(alpha=0.1))
        # else:
        #     model.add(layers.Dense(units[i], activation=activation_function, kernel_regularizer=l1_l2(l1=best_params['l1_weight'], l2=best_params['l2_weight'])))
        model.add(layers.Dense(units[i], activation='relu', kernel_regularizer=l1_l2(l1=best_params['l1_weight'], l2=best_params['l2_weight'])))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(dropouts[i]))
    
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        #optimizer=tf.keras.optimizers.get(best_params['optimizer'])(learning_rate=best_params['learning_rate']),
        optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate']),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    return model

In [ ]:
def build_final_model_lstm(best_params, input_shape):
    
    n_lstm_layers = best_params['n_lstm_layers']
    lstm_dropout = [best_params[f'lstm_dropout_{i}'] for i in range(n_lstm_layers)]
    lstm_units = [best_params[f'lstm_units_{i}'] for i in range(n_lstm_layers)]
    
    n_dense_layers = best_params['n_dense_layers']
    dense_units = [best_params[f'dense_units_{i}'] for i in range(n_dense_layers)]
    dense_dropout = [best_params[f'dense_dropout_{i}'] for i in range(n_dense_layers)]
    
    # Construção do modelo
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=input_shape))  # (look_back, n_features)
    
    # Camadas LSTM
    for i in range(n_lstm_layers):
        return_sequences = (i < n_lstm_layers - 1)  # Retorna sequências se não for a última camada
        model.add(layers.LSTM(
            lstm_units[i],
            return_sequences=return_sequences,
            kernel_regularizer=l2(best_params['l2_weight'])
        ))
        model.add(layers.Dropout(lstm_dropout[i]))
    
    # Camadas Densas
    for i in range(n_dense_layers):
        model.add(layers.Dense(dense_units[i], activation='relu'))
        model.add(layers.Dropout(dense_dropout[i]))
    
    # Saída
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Compilar
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate']),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    return model

In [ ]:
def build_final_model_cnn_lstm(best_params, input_shape):
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=input_shape))

    current_seq_length = input_shape[0]
    
    n_cnn_layers = best_params['n_cnn_layers']

    # Camadas Conv1D - Construídas com os melhores parâmetros por camada
    for i in range(n_cnn_layers):
        filters = best_params[f'cnn_filters_{i}']
        # Obtém os kernel_size e pool_size específicos desta camada do best_params
        kernel_size = best_params[f'cnn_kernel_size_{i}']
        pool_size = best_params[f'cnn_pooling_size_{i}']
        cnn_dropout_rate = best_params[f'cnn_dropout_{i}']

        # Recalcula ajustados para o modelo final, garantindo validade
        new_seq_length_after_conv = (current_seq_length - kernel_size) + 1
        adjusted_pool_size = min(pool_size, max(1, new_seq_length_after_conv))

        model.add(layers.Conv1D(
            filters=filters,
            kernel_size=kernel_size,
            activation='relu',
            padding='valid'
        ))
        model.add(layers.MaxPooling1D(pool_size=adjusted_pool_size))
        model.add(layers.Dropout(cnn_dropout_rate))

        current_seq_length = int(np.floor(new_seq_length_after_conv / adjusted_pool_size))

        # Avisa se a dimensão ficou inválida e para de adicionar CNNs se for o caso
        if current_seq_length <= 0 and i < n_cnn_layers - 1:
            print(f"Atenção: A camada Conv1D {i+1} resultou em uma dimensão de saída inválida ({current_seq_length}). As camadas CNN subsequentes foram ignoradas.")
            break


    n_lstm_layers = best_params['n_lstm_layers']
    lstm_units = [best_params[f'lstm_units_{i}'] for i in range(n_lstm_layers)]
    lstm_dropout = [best_params[f'lstm_dropout_{i}'] for i in range(n_lstm_layers)]
    
    n_dense_layers = best_params['n_dense_layers']
    dense_units = [best_params[f'dense_units_{i}'] for i in range(n_dense_layers)]
    dense_dropout = [best_params[f'dense_dropout_{i}'] for i in range(n_dense_layers)]
    # Camadas LSTM
    for i in range(n_lstm_layers):
        return_sequences = (i < n_lstm_layers - 1)
        model.add(layers.LSTM(
            lstm_units[i],
            return_sequences=return_sequences,
            kernel_regularizer=l2(best_params['l2_weight'])
        ))
        model.add(layers.Dropout(lstm_dropout[i]))

    # Camadas Densas
    for i in range(n_dense_layers):
        model.add(layers.Dense(dense_units[i], activation='relu'))
        model.add(layers.Dropout(dense_dropout[i]))

    # Saída
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Compilar
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate']),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    return model

#### Resultados

In [11]:
from sklearn.metrics import roc_curve, auc, classification_report

def evaluate_model(model, X_test, y_test, threshold=0.9897): #Limiar 0.9897
    y_prob = model.predict(X_test).ravel()

    # AUC e FPR
    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    roc_auc = auc(fpr, tpr)
    
    # Encontre o threshold ótimo (maximiza TPR - FPR)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    
    y_test_1d = y_test.squeeze()
    y_pred = (y_prob > threshold).astype(int)
    
    # Métricas
    print(classification_report(y_test_1d, y_pred))
    print(pd.crosstab(y_test_1d, y_pred, rownames=['Real'], colnames=['Predito'], margins=True))
    print(f'AUC: {roc_auc:.4f}')
    print(f'Threshold ótimo: {optimal_threshold:.4f}')
    
    return roc_auc, threshold, fpr, tpr

In [ ]:
results = {}

N_TRIALS = 200

for dataset_name, data in datasets.items(): #datasets_seq.items(): se for LSTM ou CNN + LSTM
    print(f'\n=== Processando {dataset_name} ===')

    study_name = f'tcc_auc_all_features_CV_{dataset_name.replace(" ", "_")}_v2'
    #study_name = f'tcc_auc_all_features_{dataset_name.replace(" ", "_")}_lstm_v2'
    #study_name = f'tcc_auc_all_features_{dataset_name.replace(" ", "_")}_cnn_lstm_v1'
    storage = 'sqlite:///../tcc.db'
    model_path = f'models/{study_name}.keras'

    if os.path.exists(model_path):
        print(f"Modelo {model_path} já existe. Carregando modelo...")
        best_model = tf.keras.models.load_model(model_path)

        print(f'\n=== Avaliação em {dataset_name} ===')
        roc_auc, optimal_threshold, fpr, tpr = evaluate_model(best_model, data['X_test'], data['y_test'])
        results[dataset_name] = {
            'model': best_model,
            'auc': roc_auc,
            'threshold': optimal_threshold,
            'fpr': fpr,
            'tpr': tpr
        }
        continue

    try: 
        study = optuna.load_study(study_name=study_name, storage=storage)
        print(f"Estudo {study_name} encontrado. Verificando trials...")
        
    except KeyError:
        print(f"Estudo {study_name} não existe. Criando novo estudo...")
        study = optuna.create_study(
            study_name=study_name,
            direction='maximize',
            load_if_exists=True,
            sampler=optuna.samplers.TPESampler(seed=42),
            storage=storage
        )



    if (N_TRIALS - len(study.trials)) > 0:
        print(f"Executando {N_TRIALS - len(study.trials)} trials restantes...")
        study.optimize(
            lambda trial: objective(trial, data['X_train'], data['y_train']),
            n_trials=(N_TRIALS - len(study.trials)),
            gc_after_trial=True,
        )
    else:
        print(f"Todos os trials já foram executados. Pulando otimização.")
    
    # Passo 2: Treinar o modelo final com os melhores parâmetros
    best_model = build_final_model_layer(study.best_params, data['X_train'].shape[1:]) #LSTM: build_final_model_lstm, #CNN + LSTM: build_final_model_cnn_lstm, #RNA: build_final_model_layer
    final_model_callback = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_auc',
            patience=50,
            mode='max',
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_auc',
            factor=0.5,
            patience=10,
            mode='max',
            min_lr=1e-7
        )
    ]
    
    best_model.fit(
        data['X_train'],
        data['y_train'],
        epochs=200,
        batch_size=study.best_params['batch_size'],
        validation_split=0.1,
        callbacks=final_model_callback,
        verbose=1
    )
    
    # Salvar o modelo
    print(f'\n=== Salvando o modelo em models ===')
    best_model.save(f'models/{study_name}.keras')
    # Passo 3: Avaliar no teste
    print(f'\n=== Avaliação em {dataset_name} ===')
    roc_auc, optimal_threshold, fpr, tpr = evaluate_model(best_model, data['X_test'], data['y_test'])
    results[dataset_name] = {
        'model': best_model,
        'auc': roc_auc,
        'threshold': optimal_threshold,
        'fpr': fpr,
        'tpr': tpr
    }


=== Processando Fundamental ===
Estudo tcc_auc_all_features_CV_Fundamental_v2 encontrado. Verificando trials...
Executando 172 trials restantes...
=== Iniciando Fold 1 de 5 ===


I0000 00:00:1748778236.528509 3723708 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060, pci bus id: 0000:07:00.0, compute capability: 8.9


Epoch 1/100


I0000 00:00:1748778240.332109 3738157 service.cc:152] XLA service 0x77eba80088b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748778240.332150 3738157 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4060, Compute Capability 8.9
2025-06-01 08:44:00.469718: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1748778241.075383 3738157 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/141 ━━━━━━━━━━━━━━━━━━━━ 18:18 8s/step - accuracy: 0.4688 - auc: 0.3815 - loss: 13.2070

...

=== Fim dos Folds ===
AUC Médio dos Folds: 0.8020
=== Iniciando Fold 1 de 5 ===
Epoch 1/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 13s 50ms/step - accuracy: 0.5118 - auc: 0.5547 - loss: 3.6664 - val_accuracy: 0.7959 - val_auc: 0.6803 - val_loss: 3.1659 - learning_rate: 2.4736e-04
Epoch 2/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5606 - auc: 0.6164 - loss: 3.5085 - val_accuracy: 0.7879 - val_auc: 0.6729 - val_loss: 3.1395 - learning_rate: 2.4736e-04
Epoch 3/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5502 - auc: 0.6055 - loss: 3.4765 - val_accuracy: 0.7737 - val_auc: 0.6470 - val_loss: 3.1324 - learning_rate: 2.4736e-04
Epoch 4/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5595 - auc: 0.6186 - loss: 3.3735 - val_accuracy: 0.7569 - val_auc: 0.6525 - val_loss: 3.0969 - learning_rate: 2.4736e-04
Epoch 5/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5718 - auc: 0.6422 - loss: 3.3214 - val_accuracy: 0.7657 - val_auc: 0.6644 - val_loss: 3.0600 -

[I 2025-06-01 10:49:01,934] Trial 36 finished with value: 0.8087077021598816 and parameters: {'n_layers': 4, 'units_0': 224, 'dropout_0': 0.21838762220014432, 'units_1': 96, 'dropout_1': 0.33178932434061914, 'units_2': 320, 'dropout_2': 0.4603731649176487, 'units_3': 416, 'dropout_3': 0.4650889659758929, 'learning_rate': 0.00024735915583177614, 'l2_weight': 1.1173804592233608e-05, 'l1_weight': 0.00018873591999723526, 'optimizer': 'SGD'}. Best is trial 36 with value: 0.8087077021598816.


=== Fim dos Folds ===
AUC Médio dos Folds: 0.8087
=== Iniciando Fold 1 de 5 ===
Epoch 1/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 13s 48ms/step - accuracy: 0.5462 - auc: 0.5826 - loss: 4.1822 - val_accuracy: 0.7551 - val_auc: 0.6881 - val_loss: 3.5710 - learning_rate: 0.0011
Epoch 2/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5602 - auc: 0.6318 - loss: 3.7859 - val_accuracy: 0.6096 - val_auc: 0.6917 - val_loss: 3.4590 - learning_rate: 0.0011
Epoch 3/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5544 - auc: 0.6462 - loss: 3.5391 - val_accuracy: 0.5652 - val_auc: 0.6568 - val_loss: 3.2773 - learning_rate: 0.0011
Epoch 4/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5871 - auc: 0.6800 - loss: 3.2891 - val_accuracy: 0.4614 - val_auc: 0.7009 - val_loss: 3.2201 - learning_rate: 0.0011
Epoch 5/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5637 - auc: 0.6796 - loss: 3.0880 - val_accuracy: 0.5253 - val_auc: 0.6549 - val_loss: 2.9339 - learning_rate: 

[I 2025-06-01 11:01:04,892] Trial 37 finished with value: 0.7868496298789978 and parameters: {'n_layers': 4, 'units_0': 224, 'dropout_0': 0.22169507456551027, 'units_1': 128, 'dropout_1': 0.33312413688354525, 'units_2': 320, 'dropout_2': 0.4501059238331854, 'units_3': 416, 'dropout_3': 0.46289851744194377, 'learning_rate': 0.001063617356228044, 'l2_weight': 1.007895584627717e-05, 'l1_weight': 0.00021251861275921346, 'optimizer': 'SGD'}. Best is trial 36 with value: 0.8087077021598816.


=== Fim dos Folds ===
AUC Médio dos Folds: 0.7868
=== Iniciando Fold 1 de 5 ===
Epoch 1/100
140/141 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5005 - auc: 0.5097 - loss: 6.9345

## Treinando o modelo com os melhores parâmetros do Optuna -- OLD VERSION

In [ ]:
storage_study = 'sqlite:///tcc.db'
study = optuna.load_study(
    study_name='tcc_v5',
    storage=storage_study
)
best_params = study.best_params
print('Melhores hiperparâmetros:', best_params)

### F0

In [ ]:
def test_model(X_train, y_train, X_test, y_test, dataset):
    study = optuna.load_study(
        study_name=f'tcc_{dataset}_v5',
        storage=storage_study
    )
    best_model = create_model(study.best_trial, X_train.shape[1:])
    best_model.fit(
        X_train,
        y_train,
        epochs=150,
        batch_size=best_params['batch_size'],
        verbose=1
    )

    y_test_pred = (best_model.predict(X_test)).ravel()
    test_auc = roc_auc_score(y_test, y_test_pred)

    
    print(f'AUC no teste: {test_auc:.4f}')

In [ ]:
from sklearn.metrics import classification_report
model = tf.keras.models.Sequential()
model.add(layers.InputLayer(input_shape=X_train_f0.shape[1:]))

for _ in range(best_params['n_layers']):
    model.add(layers.Dense(best_params['units'], activation='relu'))
    model.add(layers.Dropout(best_params['dropout']))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate']),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

callback = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy',
    patience=75,
    verbose=1,
)

hist = model.fit(
    X_train_f0,
    y_train_f0,
    epochs=150,
    batch_size=best_params['batch_size'],
    verbose=1,
    callbacks=[callback]
)


In [ ]:
y_prob_f0 = model.predict(X_test_f0)
y_pred_labels_f0 = (y_prob_f0 > 0.5).astype(int)
acc = accuracy_score(y_test_f0, y_pred_labels_f0)
print('Acurácia no conjunto de teste:', acc)
print(classification_report(y_test_f0, y_pred_labels_f0))

In [ ]:
y_test_1d_f0 = y_test_f0.squeeze()
y_pred_labels_1d_f0 = y_pred_labels_f0.squeeze()
y_prob_1d_f0 = y_prob_f0.squeeze()

In [ ]:
print(pd.crosstab(y_test_1d_f0, y_pred_labels_1d_f0, rownames=['Real'], colnames=['Predito'], margins=True),'')


In [ ]:
limiar = 0.9897
y_pred_limiar_f0 = (y_prob_f0 > limiar)
y_pred_limiar_f0 = np.multiply(y_pred_limiar_f0, 1)
y_pred_limiar_f0 

In [ ]:
y_pred_limiar_1d_f0 = y_pred_limiar_f0.squeeze()

In [ ]:
from sklearn.metrics import confusion_matrix

tn_f0, fp_f0, fn_f0, tp_f0 = confusion_matrix(y_test_1d_f0, y_pred_limiar_1d_f0).ravel()

taxa_fp_f0 = fp_f0 / (fp_f0 + tn_f0)
taxa_detec_f0 = tp_f0 / (tp_f0 + fn_f0)

print('Taxa de Falsos Positivos:', taxa_fp_f0)
print('Taxa de Detecção:', taxa_detec_f0)

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr_f0, tpr_f0, thresholds_f0 = roc_curve(y_test_1d_f0, y_prob_f0)
roc_auc_f0 = auc(fpr_f0, tpr_f0)
print('AUC:', roc_auc_f0)

In [ ]:
import matplotlib.pyplot as plt

def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)
    plt.grid(True)

plt.figure(figsize=(8, 6))
plot_roc_curve(fpr_f0, tpr_f0)
plt.show()


### F0 e 2F0

In [ ]:
from sklearn.metrics import classification_report
model = tf.keras.models.Sequential()
model.add(layers.InputLayer(input_shape=X_train_h2.shape[1:]))

for _ in range(best_params['n_layers']):
    model.add(layers.Dense(best_params['units'], activation='relu'))
    model.add(layers.Dropout(best_params['dropout']))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate']),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

callback = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy',
    patience=75,
    verbose=1,
)

hist = model.fit(
    X_train_h2,
    y_train_h2,
    epochs=150,
    batch_size=best_params['batch_size'],
    verbose=1,
    callbacks=[callback]
)




In [ ]:
y_prob_h2 = model.predict(X_test_h2)
y_pred_labels_h2 = (y_prob_h2 > 0.5).astype(int)
acc = accuracy_score(y_test_h2, y_pred_labels_h2)
print('Acurácia no conjunto de teste:', acc)
print(classification_report(y_test_h2, y_pred_labels_h2))

In [ ]:
y_test_1d_h2 = y_test_h2.squeeze()
y_pred_labels_1d_h2 = y_pred_labels_h2.squeeze()
y_prob_1d_h2 = y_prob_h2.squeeze()

In [ ]:
print(pd.crosstab(y_test_1d_h2, y_pred_labels_1d_h2, rownames=['Real'], colnames=['Predito'], margins=True),'')


In [ ]:
limiar = 0.9897
y_pred_limiar_h2 = (y_prob_h2 > limiar)
y_pred_limiar_h2 = np.multiply(y_pred_limiar_h2, 1)
y_pred_limiar_h2 

In [ ]:
y_pred_limiar_1d_h2 = y_pred_limiar_h2.squeeze()

In [ ]:
from sklearn.metrics import confusion_matrix

tn_h2, fp_h2, fn_h2, tp_h2 = confusion_matrix(y_test_1d_h2, y_pred_limiar_1d_h2).ravel()

taxa_fp_h2 = fp_h2 / (fp_h2 + tn_h2)
taxa_detec_h2 = tp_h2 / (tp_h2 + fn_h2)

print('Taxa de Falsos Positivos:', taxa_fp_h2)
print('Taxa de Detecção:', taxa_detec_h2)

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr_h2, tpr_h2, thresholds_h2 = roc_curve(y_test_1d_h2, y_prob_h2)
roc_auc_h2 = auc(fpr_h2, tpr_h2)
print('AUC:', roc_auc_h2)

In [ ]:
import matplotlib.pyplot as plt

def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)
    plt.grid(True)

plt.figure(figsize=(8, 6))
plot_roc_curve(fpr_h2, tpr_h2)
plt.show()


### F0, 2F0 e 3F0

In [ ]:
from sklearn.metrics import classification_report
model = tf.keras.models.Sequential()
model.add(layers.InputLayer(input_shape=X_train.shape[1:]))

for _ in range(best_params['n_layers']):
    model.add(layers.Dense(best_params['units'], activation='relu'))
    model.add(layers.Dropout(best_params['dropout']))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate']),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

callback = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy',
    patience=75,
    verbose=1,
)

hist = model.fit(
    X_train,
    y_train,
    epochs=150,
    batch_size=best_params['batch_size'],
    verbose=1,
    callbacks=[callback]
)


In [ ]:
y_prob = model.predict(X_test)
y_pred_labels = (y_prob > 0.5).astype(int)
acc = accuracy_score(y_test, y_pred_labels)
print('Acurácia no conjunto de teste:', acc)
print(classification_report(y_test, y_pred_labels))

In [ ]:
y_test_1d = y_test.squeeze()
y_pred_labels_1d = y_pred_labels.squeeze()
y_prob_1d = y_prob.squeeze()

In [ ]:
print(pd.crosstab(y_test_1d, y_pred_labels_1d, rownames=['Real'], colnames=['Predito'], margins=True),'')


In [ ]:
limiar = 0.9897
y_pred_limiar = (y_prob > limiar)
y_pred_limiar = np.multiply(y_pred_limiar, 1)
y_pred_limiar 

In [ ]:
y_pred_limiar_1d = y_pred_limiar.squeeze()

In [ ]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(y_test_1d, y_pred_limiar_1d).ravel()

taxa_fp = fp / (fp + tn)
taxa_detec = tp / (tp + fn)

print('Taxa de Falsos Positivos:', taxa_fp)
print('Taxa de Detecção:', taxa_detec)

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, thresholds = roc_curve(y_test_1d, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

In [ ]:
import matplotlib.pyplot as plt

def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)
    plt.grid(True)

plt.figure(figsize=(8, 6))
plot_roc_curve(fpr, tpr)
plt.show()


# Métodos Tradicionais

In [ ]:
X_test_rec = pd.read_csv('f0/X_test_rec_f0.csv')
y_test_f0 = pd.read_parquet('f0/y_test_f0.parquet')

X_test_rec_nh = X_test_rec.iloc[:, :500].values.astype(np.complex128)

In [ ]:
X_test_rec_nh

In [ ]:
from sklearn.metrics import roc_curve

M = 500;
alfa=0.05;

teta = np.angle(X_test_rec_nh)
y_test_only_1=np.flatnonzero(y_test)
y_test_only_0=np.flatnonzero(y_test==0)
## CSM
CSM=(np.power((1/M)*np.sum(np.cos(teta),axis=1),2))+(np.power((1/M)*np.sum(np.sin(teta),axis=1),2))

from scipy.stats.distributions import chi2

VC_CSM=chi2.ppf(1-alfa, df=2)/(2*M)
VC_CSM
y_CSM_txd=(CSM[y_test_only_1] > VC_CSM)
#y_pred_limiar=int(y_pred_limiar)

# Converting boolean to integer
y_CSM_txd = np.multiply(y_CSM_txd, 1)

y_CSM_txd=np.mean(y_CSM_txd)


y_CSM_fp=(CSM[y_test_only_0] > VC_CSM)
#y_pred_limiar=int(y_pred_limiar)

# Converting boolean to integer
y_CSM_fp = np.multiply(y_CSM_fp, 1)

y_CSM_fp=np.mean(y_CSM_fp)

y_CSM=(CSM > VC_CSM)
#y_pred_limiar=int(y_pred_limiar)

# Converting boolean to integer
y_CSM = np.multiply(y_CSM, 1)

fpr_CSM, tpr_CSM, thresholds_CSM = roc_curve(y_test,CSM.transpose())
print(y_CSM_txd)
print(y_CSM_fp)
## MSC
MSC=(np.abs(np.sum(X_test_rec_nh,axis=1))**2)/(M*np.sum((np.abs(X_test_rec_nh)**2),axis=1))
MSC = np.reshape(MSC,(1,len(MSC)))

VC_MSC=1-(alfa**(1/(M-1)))
VC_MSC
y_MSC_txd=(MSC[0,y_test_only_1] > VC_MSC)
#y_pred_limiar=int(y_pred_limiar)

# Converting boolean to integer
y_MSC_txd = np.multiply(y_MSC_txd, 1)

y_MSC_txd=np.mean(y_MSC_txd)

from numpy import matlib

y_MSC_fp=(MSC[0, y_test_only_0] > np.matlib.repmat(VC_MSC,1096,1))
#y_pred_limiar=int(y_pred_limiar)

# Converting boolean to integer
y_MSC_fp = np.multiply(y_MSC_fp, 1)

y_MSC_fp=np.mean(y_MSC_fp)

fpr_MSC, tpr_MSC, thresholds_MSC = roc_curve(y_test,MSC.transpose())


In [ ]:
# Supondo que você já tenha as curvas externas:
results['CSM'] = {
    'fpr': fpr_CSM, 
    'tpr': tpr_CSM, 
    'auc': auc(fpr_CSM, tpr_CSM),
    'label': 'CSM'
}

results['MSC'] = {
    'fpr': fpr_MSC, 
    'tpr': tpr_MSC, 
    'auc': auc(fpr_MSC, tpr_MSC),
    'label': 'MSC'
}

# Curva ROC - Comparação

In [ ]:
def plot_all_roc_curves(results):
    plt.figure(figsize=(10, 8))
    
    # Plotar cada curva do dicionário
    for key in results:
        data = results[key]
        plt.plot(
            data['fpr'], 
            data['tpr'], 
            linewidth=2,
            label=f"{data.get('label', key)} (AUC = {data['auc']:.2f})"
        )
    
    # Linha de referência
    plt.plot([0, 1], [0, 1], 'k--', linewidth=2)
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title('Comparação de Curvas ROC', fontsize=14)
    plt.legend(loc='lower right', fontsize=10)
    plt.grid(True)
    plt.show()

In [ ]:
# Após o loop de avaliação dos datasets, adicione:
plot_all_roc_curves(results)